In [37]:
import pandas as pd
from enum import Enum

In [38]:
class Status(Enum):
    AVAILABLE = "available"
    DEPLOYED = "deployed"
    UNDER_MAINTENANCE = "under maintenance"

class Skill(Enum):
    BASIC = 1
    INTERMEDIATE = 2
    ADVANCED = 3

In [39]:
class RescueVehicle():
    def __init__(self, id: str, max_persons: int, max_num_teams: int):
        self.id = id
        self.max_persons = max_persons
        self.max_num_teams = max_num_teams
        self.num_victims = 0
        self.num_teams = 0
        self.curr_persons = self.num_victims+self.num_teams
    
    def get_id(self):
        return self.id
    
    def get_max_persons(self):
        return self.max_persons
    
    def get_curr_persons(self):
        return self.num_victims+self.num_teams
    
    def add_persons(self, num_persons: int, is_team: bool = True):
        assert num_persons+self.curr_persons <= self.max_persons, "add_persons(): not enough space"
        if is_team:
            assert self.num_teams+num_persons > self.max_num_teams, "add_persons(): too much teams"
            self.num_teams += num_persons
        else:
            self.num_victims += num_persons
            self.curr_persons += num_persons
    
    def remove_persons(self, num_persons: int, is_team: bool = True):
        assert self.curr_persons-num_persons < 0, "remove_persons(): invalid value"
        if is_team:
            assert self.num_teams-num_persons < 0, "remove_persons(): invalid value teams"
            self.num_teams -= num_persons
        else:
            assert self.num_victims-num_persons < 0, "remove_persons(): invalid value victims"
            self.num_victims -= num_persons
            self.curr_persons -= num_persons
        

        
class RescueBoat(RescueVehicle):
    def __init__(self):
        super().__init__(4, 2)

class Helicopter(RescueVehicle):
    def __init__(self):
        super().__init__(6, 2)
    

In [40]:
class Location():
    def __init__(self, long: float, lat: float):
        self.long = long
        self.lat = lat
    
    def get_location(self):
        return self.long, self.lat
    
    def set_location(self, long: float, lat: float):
        self.long = long
        self.lat = lat

class Personnel():
    def __init__(self, id: str, skill: Skill):
        self.id = id
        self.skill = skill
        self.team_id = None
    
    def get_id(self):
        return self.id
    
    def get_skill(self):
        return self.skill
    
    def get_team_id(self):
        return self.team_id

    def is_assigned(self):
        return self.team_id != None
    
    def set_team_id(self, team_id: str):
        self.team_id = team_id

class Team():
    def __init__(self, id: str, location: Location):
        self.id = id
        self.location = location
        self.personnels = {}
        self.status = Status.AVAILABLE
        
    def get_id(self):
        return self.id
    
    def get_location(self):
        return self.location.get_location()
    
    def get_resource(self):
        return self.resource
    
    def is_personnel_assigned(self, personnel_id: str):
        return personnel_id in self.personnels
    
    def get_status(self):
        return self.status
       
    def get_all_personnels(self):
        return list(self.personnels.values())
            
    def get_personnel(self, personnel_id: str):
        assert personnel_id in self.personnels, f"get_personnel(): personnel_id not found in team {self.id}"
        return self.personnels[personnel_id]
    
    def get_skill(self):
        skill_map = {1: Skill.BASIC, 2: Skill.INTERMEDIATE, 3: Skill.ADVANCED}
        personnels = self.get_all_personnels()
        avg_skill = int(sum([p.get_skill().value for p in personnels])/len(personnels))
        return skill_map[avg_skill]
    
    def get_number_personnels(self):
        return len(self.personnels)
    
    def set_status(self, status: Status):
        self.status = status
    
    def set_location(self, long: float, lat: float):
        self.location.set_location(long, lat)
        
    def add_personnel(self, personnel: Personnel):
        self.personnels.update({personnel.get_id(): personnel})
        
    def remove_personnel(self, personnel_id: str):
        assert personnel_id in self.personnels, f"remove_personnel(): personnel_id not found in team {self.id}"
        self.personnels = self.personnels.pop(personnel_id)

    
    
class FirePlace():
    def __init__(self, name: str, location: Location):
        self.name = name
        self.location = location
        self.rescue_vehicles = []
        self.map_teams_to_rescue_vehicles = {}
        self.personnels = {}
        self.teams = {}
        
    def get_name(self):
        return self.name
    
    def get_location(self):
        return self.location.get_location()
    
    def get_resource(self):
        return self.resource
    
    def get_personnel(self, personnel_id: str):
        assert personnel_id in self.personnels, f"get_personnel(): personnel_id not found in fire place {self.name}"
        return self.personnels[personnel_id]
    
    def get_team(self, team_id: str):
        assert team_id in self.teams, f"get_team(): team_id not found in fire place {self.name}"
        return self.teams[team_id]
    
    def get_all_personnels(self):
        return list(self.personnels.values())
    
    def get_all_teams(self):
        return list(self.teams.values())
    
    def get_number_personnels(self):
        return len(self.personnels)
    
    def get_number_teams(self):
        return len(self.teams)
    
    def get_number_available_teams(self):
        return len([t for t in self.get_all_teams() if t.get_status()==Status.AVAILABLE])
    
    def get_available_teams(self):
        return [t for t in self.get_all_teams() if t.get_status()==Status.AVAILABLE]
    
    def add_vehicle(self, vehicle: RescueVehicle):
        self.rescue_vehicles.append(vehicle)
        
    def remove_vehicle(self, id: str):
        self.rescue_vehicles = [v for v in self.rescue_vehicles if v.get_id()!=id]
    
    def add_personnel(self, personnel: Personnel):
        self.personnels.update({personnel.get_id(): personnel})
        
    def remove_personnel(self, personnel_id: str):
        assert personnel_id in self.personnels, f"remove_personnel(): personnel_id not found in fire place {self.name}"
        self.personnels = self.personnels.pop(personnel_id)
        
    def add_team(self, team: Team):
        assert team.get_location() == self.get_location(), "location needs to be equal"
        self.teams.update({team.get_id(): team})
        
    def remove_team(self, team_id: str):
        assert team_id in self.teams, f"remove_team(): team_id not found in fire place {self.name}"
        self.teams = self.teams.pop(team_id)
        

In [41]:
import math
def haversine_distance(lon1, lat1, lon2, lat2):
 
    """
    Calculate the great-circle distance (in kilometers) between two points 
    on the Earth's surface given their latitude and longitude in degrees.
    """
    # Convert latitude and longitude from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    # Haversine formula
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    
    # Radius of the Earth in kilometers
    R = 6371.0
    
    # Calculate the distance
    distance = R * c
    
    return distance

In [42]:
class Manager():
    
    def __init__(self):
        self.fireplaces = []
        
    def add_fireplace(self, fireplace: FirePlace):
        self.fireplaces.append(fireplace)
    
    def remove_fireplace(self, location: Location):
        self.fireplaces = [fp for fp in self.fireplaces if fp.get_location() == location.get_location()]
    
    def get_available_teams(self)->dict:
        return {fp.get_location(): fp.get_available_teams() for fp in self.fireplaces}
     
    def update_status(self, teams: list[Team], status: Status):
        for team in teams:
            team.set_status(status)
            
    def update_location(self, teams: list[Team], location: Location):
        long, lat = location.get_location()
        for team in teams:
            team.set_location(long, lat)
        
    def deploy(self, location: Location, required_skill: Skill, min_team_size: int)->list[Team]:
        all_available_teams = self.get_available_teams()
        all_available_teams = {haversine_distance(*(list(loc)+list(location.get_location()))): teams for loc, teams in all_available_teams.items()}
        closest_team_location = sorted(list(all_available_teams.keys()))
        curr_team_size = 0
        deployed_teams = []
        for loc in closest_team_location:
            if curr_team_size >= min_team_size:
                break
            available_teams = all_available_teams[loc]
            for team in available_teams:
                if team.get_skill().value >= required_skill.value:
                    deployed_teams.append(team)
                    curr_team_size+=team.get_number_personnels()
        self.update_status(deployed_teams, Status.DEPLOYED)
        self.update_location(deployed_teams, location)
        return deployed_teams     

In [43]:
p1 = Personnel('p1', Skill.BASIC)
p2 = Personnel('p2', Skill.ADVANCED)
p3 = Personnel('p3', Skill.INTERMEDIATE)

p4 = Personnel('p4', Skill.BASIC)
p5 = Personnel('p5', Skill.ADVANCED)
p6 = Personnel('p6', Skill.INTERMEDIATE)

t1 = Team('t1', Location(5,5))
t2 = Team('t2', Location(10,10))
t3 = Team('t3', Location(10,10))

fp1 = FirePlace('fp1', Location(5,5))
fp2 = FirePlace('fp2', Location(10,10))

manager = Manager()

t1.add_personnel(p1)
t1.add_personnel(p2)
t1.add_personnel(p3)
t2.add_personnel(p4)
t3.add_personnel(p5)
t3.add_personnel(p6)

fp1.add_team(t1)
fp2.add_team(t2)
fp2.add_team(t3)

manager.add_fireplace(fp1)
manager.add_fireplace(fp2)

In [44]:
deployed_teams = manager.deploy(Location(4,4), Skill.BASIC, min_team_size=5)
[dp.get_id()for dp in deployed_teams]

['t1', 't2', 't3']